# UC Davis | STA160 Group 7 | Midterm Project

### Checklist

**Data Cleaning**
- Take out too expensive prices  **DONE**
    - look at the info when doing this 
- Take out zero avalibility **DONE**
    -   check other stuff
-   Any reviews in 2019
-   look into cleaning the listings with 0 reviews **DONE**

**EDA**
- Basic Graphs visiualizing 
    - prices 
    - neghborhoods 
    - density/heat map
- Ivan 
- Rishi

**NLP**
- Cleaning on name column
- Vishnu
- ADI
- Go Into Graph

**Write the interpretation**
- Ivan 
- Rishi


**Given Time** 
- Explore other research questions
    - such as housing data

## Group Info

UC DAVIS STA 160 Group 7

Group Members 

- Vishnu Rangiah, 916562849, vrrangiah@ucdavis.edu
- Aditya Kallepalli, 915079375, arkallepalli@ucdavis.edu
- Rishi Bhuva, 915218518, rdbhuva@ucdavis.edu
- Ivan Yang, 915463046 igyang@ucdavis.edu


## Abstract

- 200 word abstract

## Introduction

In this report we will be analyzing Airbnb data from the Airbnb open new york city 2019 data set. 


Airbnb is a platform that operates an online marketplace for lodging, primarily homestays for vacation rentals, and tourism activities. This platform is accessible through website or mobile app. 

The data we are currently analyzing comes from : https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

This dataset describes listing activity and metrics in New York City, New York for 2019. Through this dataset, we can get information regarding hosts, geographical availability and necessary metrics needed to make predictions and draw conclusions. 

We will be performing our analysis using Python with the packages xxxxxx . In Section II, we will be cleaning, and preforming exploratory analysis on the data. Then we will explore the relationship between the names of each listing and the popularity of the listing using Natural Language Processing Methods in Section III.

[//]: attachment:./# (We will be cleaning, and preforming exploratory analysis on the data. 
Then we will explore the realtionship between the Names of each listing and the popularity of the listing using Natural Language Processing Methods.)


#### NEXT STEPS: EDA, NLP, MODELS, CONCLUSION 

## Importing Data Set

In [ ]:
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
import re
import more_itertools as more
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


df = pd.read_csv('Airbnb_NYC_2019.csv')

In [ ]:
!pip install dataframe_image

In [ ]:
df

# Our original Dataset
# Explain dataset, possibly provide graphs to display data more properly

Here we find that our dataset contains 48,895 rows which represent indivual listings in New York. We are also provided with 16 different columns which contain necessary factors needed for exploratory data analysis. 

We can see these different columns below:

In [ ]:
for i in df.columns:
    print(i)

## No. of Variables + Entries, Variable Names, and Data Types:

Here, we can see that our dataframe consists of 48,895 rows and 16 columns.

In [ ]:
df.shape

Data Types of Each Column

A brief analysis as to what data type each column is can be seen below:

In [ ]:
df.dtypes

## Data Cleaning  

- Find outliers 
- Get rid of listing that are too expensive 
    - Why is the price so high: super bowl?
- Get rid of listing that do not have avalibility for 2019
- Possibly find potential negative and NA values 

- The Airbnb calendar for a listing does not differentiate between a booked night vs an unavailable night, therefore these bookings have been counted as "unavailable". This serves to understate the Availability metric because popular listings will be "booked" rather than being "blacked out" by a host.

- avaliblity_365 :  number of days the property is avalibly for rent 
    - Data shows alot of properties of 0 days avalible 
        - Why was it 0: 
            - host didnt want to rent or host took off listing but it was still on sight 
            - airbnb suspended listing 

Our first line of action when analyzing this data is to properly clean the dataset. Having clean data will provide us with the highest quality of information needed and therefore will provide us with the most accurate predictions and correlations. Our process for cleaning the dataset can be seen below:

We will first check the number of rows with missing values. We will then remove such rows depending on the missing variables.

In [ ]:
#Checking for null values
df.isnull().sum()

As we can see, there are 4 variables with missing values. However, we've determined that all of these missing variables do not greatly affect our NLP modelling. Therefore, we've decided to keep these listings.

On the other hand, while combing through our main dataframe, we discovered that there are a number of listings with 0 available days to be rented as well as 0 total reviews.

In [ ]:
df[df.availability_365 == 0] # Elaborate on why zero availabilty would occur/ show possible plots

The dataframe above consists of listings with zero availability. We chose to extract these listings from the dataframe we will analyze, because zero availablity means that the listings is not available at all. Therefore, this data would be irrelevant as we are want to perform analysis on listings that have availablity. 

In [ ]:
df[df.number_of_reviews == 0] # Elaborate on why some number of reviews would be zero / show possible plots

#We will remove entries with 0 days of availabilities and 0 reviews.
#We believe that these are faulty listings that were not completely removed from the AirBnB server. 
#Most of these listings were not available under the main English website but were viewable in different languages such as Spanish.

The dataframe above consists of listings with zero number of reviews. We chose to extract these values from the data we want to analyze, because number of reviews is a factor that can tell us a lot about the popularity of the listing and is an important component to our Natural Language Processing Method which we will perform. Therefore, listings with zero number of reviews would be irrelevant data and not needed for our type of analysis.



Due to the drawbacks discussed regarding 0 days of availabilities and 0 number of reviews, it would be best to simply remove these entries in the dataset we want to work with. Below, we properly remove these listings.

In [ ]:

df = df[df.availability_365 != 0] # Remove entries with 0 days of availabilities

In [ ]:
df = df[df.number_of_reviews != 0] # Remove entries with 0 number of reviews
#df = df[df.price != 0] # Remove entries that consist of $0 price
df.shape 

Above, we can see the dimensions of our reduced dataframe, excluding values of avaiablity and number of reviews being 0. 

Now, we will examine the price range of all AirBnB listings by separating these listings into quartiles to get a general insight for this column alone. Along with the 25, 50, 75 % quartile calculations, we can also get information such as the mean, standard deviation and the minimum and maximum value of our price category.

In [ ]:

quartiles = df[["price"]].describe()
quartiles

#From the initial quartile calculation, we see that there are listings with $0 as price. 
#Inspecting listings with `price` = 0

As observed in our quartile table above, there are interestingly listings that are $0. Our next course of action would be to pull up these listings and do further research.  

In [ ]:

df.loc[lambda df: df.price == 0]  # Possibly explain why some listings in our data would be $0

After multiple online searches of these listings, we've come up with mixed results of these listings' robustness. Listings such as Kimberly's seem to have been removed while all three of Adeyemi's listings are still up on AirBnB and running in 2021. Additionally, there isn't any way that we can revisit AirBnB's 2019 websites to verify these listings.

One possible reason for these $0 prices may have been a deliberate act by the hosts to temporarily remove the listing from the AirBnB market when these listings were webscraped. The additional fact that among these 0 USD listings, 3 and 2 of them belong to the same host may further indicate that it is a host-activated anomaly. 

Either way, we will remove these listings just to minimize any possibility of accruing errors based on unknown anomalies.

Below, is the process of removing entries with the price = 0.

In [ ]:
df = df[df.price != 0] # Remove listings that have $0 price.
quartiles = df[["price"]].describe() # See the statistics of price to further analyze
quartiles

Above, we can see our new summary statistics with $0 of price removed.

In [ ]:
plt.boxplot(x = df['price'], vert = False)

We can see that the data is quite skewed with multiple extremely high prices heavily affecting the data. Further research also shows that some of these data points are faulty listings, such as a 2012 Superbowl private room listing.

We will use the 1.5 Interquartile Range Test to determine outliers and store these listings in a different dataframe for further data analysis. 

In [ ]:
#Using the 1.5 Interquartile Range Test to determine outliers

IQR = np.percentile(df.price, 75)-np.percentile(df.price, 25)
lower_limit = np.percentile(df.price, 25)- 1.5*IQR
upper_limit = np.percentile(df.price, 75) + 1.5*IQR
print(lower_limit, upper_limit) 
# Calculations for outliers within the price entry of data, used to deem what is considered "too expensive."

Here, through the IQR test, we can calculate for outliers within the price entry of data. We can see that anything above the price of 332.5 dollars would be considered too expensive, and therefore would be an outlier.  Since there technically cannot be any negative values for prices, we will fix the lower limit at 10 dollars which is the minimum value as seen in our quartile table.

In [ ]:
#df

#Since there technically cannot be any negative values for prices, we will fix the lower limit at $10 which is the minimum value as seen in our quartile table.

outliersdf = df.loc[lambda df: df.price > upper_limit]
outliersdf # Dataset of what is considered too expensive, conduct plots to see analysis.
# Possibly explain why these listings would be considered expensive.

# could also use df.loc[lambda df: df.price >=6000]

#Do futher processing here later 
#df_1 = df.loc[lambda df: df.price <= 6000] # df without outliers
#df_1

outliersdf.shape

### Tidied Dataset

Below, we have cut all outliers and properly cleaned the dataset and can move on to exploratory data analysis. We can also see the dimensions of our cleaned up dataset.

In [ ]:
tidydf = df.loc[lambda df: df.price <= upper_limit]
tidydf.shape

In [ ]:
plt.figure(figsize = (12,12))
sns.heatmap(tidydf.corr(),annot=True) # Explain corr plot.

Here, we can see a proper correlation matrix between the variables within our dataset. According to this plot, we can see that the more lighter the color, the higher the correlation between each variable. Vice versa, we can see that the darker the number, the lower the correlation. From this plot, we can immediately see a high correlation of 0.6 between id and host_id. Along with this, it can be seen that number of reviews and reviews per month also have a higher correlation than others at 0.49. Along with positive correlations, we can also see negative correlations with id and number of reviews at -0.45 and with id and reviews per month at -0.45. We can also see a negative correlation between price and longitude at -0.3, although a positive correlation of 0.051 between price and latitude. 

As id is a specific tag per listing, host_id is also a specific tag per host. Therefore, this positive correlation does seem to make sense as some hosts would have multiple listings under their name. Number of reviews and reviews per month also tend to have a positive correlation as each number of review contributes to a review per month and vice versa, therefore these two variables seem to have a direct relationship. 

The lowest correlation we can see in this plot can be seen between both variables of reviews and id. The large negative correlation between them does seem accurate, as each id is specific and randomly generated and does not relate at all to reviews at all. Another negative correlation that was interesting comes from price and longitude. As longitude tends to be west and east, it can be seen that pricein New York does not tend to have any sort of relationship with those specific directions. Although, price and latitude show a somewhat positive correlation, meaning that price is more correlated with listings that range from the north and south rather than the west and east. 

In [ ]:
#do we need this entire plot or plot boxplots for specific columns?
plt.figure(figsize = (12,12))
tidydf.boxplot()

## Graphs

- Price (whole dataset, by borough, by types of room) **DONE**
- Minimum Nights (whole dataset, by borough) **DONE**
- Types of Room (whole dataaset, by borough) **DONE**
- Reviews per month (whole dataset, by borough) **DONE**
- availability 365 (whole dataset, by borough)


### If we have time 
- Some sort of graphs and explaination of popularity 
- recommended by prof: roc curve

### Extra
- heat map 
- box plot 
- how much people are making 
- which hosts make the most
- observe which neighborhoods have certain amount of avalibilty 

We will plot some graphs with our tidied dataset to gain further insight, as well as to visualize any more abnormal data points for further cleaning. We will not be subsetting these outliers from the main tidied dataset as these variables will not be our main focus for our NLP analysis. However, we will generate graphs of these outliers for further analysis.

In [ ]:
sns.set(style="darkgrid")
sns.countplot(y = 'room_type', data= tidydf)

We can see that most listings are either private rooms or the entirety of the property. This indicates that most AirBnB customers may be looking for private accomodation rather than shared spaces.

In [ ]:
sns.countplot(y = 'neighbourhood_group', data= tidydf).set_title("Listings by Borough")

Most listings are either in the boroughs of Brooklyn or Manhattan. This is logical given that these 2 boroughs contain the most attractions and activites in New York City. Staten Island has the lowest number of listings, which may be due to its geographical location being relatively inaccessible compared to the other boroughs.

In [ ]:
dfn = df['neighbourhood'].value_counts()
sns.countplot(y='neighbourhood',data=tidydf, order=dfn.iloc[:10].index).set_title('Top 10 Neighborhoods of Listings')

The top 4 neighborhoods of AirBnB listings are all in Brooklyn, with Bedford-Stuyvesant being the top neighborhood by quite a margin. The 5th to 8th most popular neighborhoods are in Manhattan. This may be due to affordability of Brooklyn compared to Manhattan. 

Density Map of All Listings According to Price

<https://geopandas.org/gallery/create_geopandas_from_pandas.html>

In [ ]:
!pip install contextily

In [ ]:
#gdf = gpd.GeoDataFrame(
    #tidydf, geometry=gpd.points_from_xy(tidydf.longitude, tidydf.latitude))

#ax = gdf.plot(figsize=(15, 15), alpha=0.5, edgecolor='k')
#ctx.add_basemap(ax, zoom = 18)
#ax.set_axis_off()

In [ ]:
import geopandas as gpd
import contextily as ctx

plt.figure(figsize=(15,30))
sns_map = sns.scatterplot(x='longitude', y='latitude', hue='price',s=20, data=tidydf)
ctx.add_basemap(sns_map, crs = 'EPSG:4326', source=ctx.providers.CartoDB.Positron)
sns_map.set_axis_off()

From the density map above, we can see that most of the listings are concentrated around the boroughs of Brooklyn and Manhattan. We can also observe that Manhattan has a higher density of expensive listings, and apart from standout anomalies, prices tend to be lower as listings become further from Manhattan.

Graphs in relation to price

In [ ]:
#sns.boxplot(x='price', data=df)
px.histogram(tidydf, x = 'price', title = 'Price Distribution')

From the histogram above, we can see a left-skewed unimodal distribution. A majority of the listings hover around a price between 35 to 130 USD. However, there are significant spikes at 150, 200, 250, and 300 USD bins, as well as smaller spikes at 175, 225, 275, and 325 USD bins. We believe that this may be caused by human psychology where hosts may round their prices to the nearest 25 or 50 dollar for a more "aesthetically pleasing" price number.

In [ ]:
#by room type and borough
sns.catplot(x='neighbourhood_group', y='price', data = tidydf, hue = 'room_type')

There is an obvious trend that shared rooms are the cheapest option in all boroughs, followed by private room and entire property. Additionally, Staten Island and the Bronx have a lot less listings above 150USD compared to Brooklyn and Manhattan. Additionally, as seen in our density map, Manhattan has the highest density of expensive listings compared to other boroughs.

In [ ]:
plt.figure(figsize=(15,30))
sns_map_out = sns.scatterplot(x='longitude', y='latitude', hue='price',s=20, data=outliersdf)
ctx.add_basemap(sns_map_out, crs = 'EPSG:4326', source=ctx.providers.CartoDB.Positron)
sns_map_out.set_axis_off()

In [ ]:
#learning about outlying prices according to room type
sns.catplot(x='room_type', y='price', data = outliersdf)

Most of the outlying expensive listings are either for entire properties or private rooms. Interestingly, the most expensive listing is a 10000USD private room in Queens. This is listing is now defunct, but we can infer that it might be a long term rental situation as the listing also indicates a minimum night requirement of 100 nights.

In [ ]:
#learning about outliers according to neighborhood and room type
sns.catplot(x='neighbourhood_group', y='price', data = outliersdf, hue = 'room_type')

Graphs in relation to minimum nights

In [ ]:
#sns.boxplot(x='minimum_nights', data=df)
px.histogram(tidydf, x = 'minimum_nights', title = 'Minimum Nights Distribution')

In [ ]:
np.mean(tidydf.minimum_nights)

Most listings in 2019 NYC have a minimum night requirement between 1 to 7 nights, with an average minimum night requirement of 6.64 and 2 nights being the hightest. There is an apparent spike of listings with a minimum night requirement of 30 nights.

There seems to be multiple outliers, with one as extreme as 1250 minimum nights. This wide range of minimum night requirements allude to the existence of not just long term vacation rental properties but also potentially long term residence.

We will determine their outliers again using the 1.5 IQR test.

In [ ]:
#Using the 1.5 Interquartile Range Test to determine outliers

IQR_mn = np.percentile(tidydf.minimum_nights, 75)-np.percentile(tidydf.minimum_nights, 25)
lower_limit_mn = np.percentile(tidydf.minimum_nights, 25)- 1.5*IQR
upper_limit_mn = np.percentile(tidydf.minimum_nights, 75) + 1.5*IQR
print(lower_limit_mn, upper_limit_mn) 
# Calculations for outliers within the price entry of data, used to deem what is considered "too expensive."

In [ ]:
#Since there technically cannot be any negative values for min nights, we will fix the lower limit at 0.

outliersdf_mn = tidydf.loc[lambda tidydf: tidydf.minimum_nights > upper_limit_mn]
outliersdf_mn # Dataset of what is considered too expensive, conduct plots to see analysis.
# Possibly explain why these listings would be considered expensive.

# could also use df.loc[lambda df: df.price >=6000]

#Do futher processing here later 
#df_1 = df.loc[lambda df: df.price <= 6000] # df without outliers
#df_1

outliersdf_mn.shape

In [ ]:
tidydf_mn = tidydf.loc[lambda tidydf: tidydf.minimum_nights <= upper_limit_mn]
tidydf_mn.shape

In [ ]:
sns.catplot(x='neighbourhood_group', y='minimum_nights', data = tidydf_mn, hue='room_type')

From the graph, it seems like Staten Island and the Bronx are not popular boroughs for long term rentals longer than 30 days. Further investigation of the few orange anomalies in the Bronx reveal that they are all from the same host, Sasha (Host ID: 2988712), who seems to have multiple 90-day listings around Claremont Village and Mount Hope in the Bronx.

Most longer term rentals in Brooklyn, Manhattan, and Queens are of entire properties. Interesting anomalies include the 90 day minimum night requirement listings for a shared room in Manhattan. Further investigation shows that these listings (Host IDs: 21628183 and 23184420) are both from LaGuardia Houses Public Housing Development looking for long term housemates. We believe that these listings are a result of hosts trying to bypass New York's legal barrier of subletting public housing for additional income, adding onto the high accessibility of AirBnB for anyone to put out a listing.

In [ ]:
sns.catplot(x='neighbourhood_group', y='minimum_nights', data = outliersdf_mn, hue='room_type')

As expected, most extreme values of minimum night requirement are from listings in Manhattan and Brooklyn, with Queens and the Bronx each only having 1 listing and Staten Island not having any. Both listings in Queens and the Bronx seem to be long term rental leasings, just like most of the other leasings in this plot.

Graph by Reviews Per Month

In [ ]:
px.histogram(tidydf, x = 'reviews_per_month', title = 'Reviews Per Month Distribution')

In [ ]:
np.mean(tidydf.reviews_per_month)

The number of reviews are heavily skewed to the left with listings getting an average of 1.83 reviews per month. There are obvious outliers, with a listing getting as many as 58.5 average reviews per month.

In [ ]:
sns.catplot(x='neighbourhood_group', y='reviews_per_month', data = tidydf, hue='room_type')

We can see that most boroughs have a similar pattern of reviews per month, with 2 anomalies in Manhattan. Further analysis reveals that both listings (ID: 32678719 and 32678720) are by the same host Row NYC (Host ID: 244361589) which is a hotel in New York City. They may have more reviews as they may be more attractive to customers as an established hotel chain. Additionally, they may have multiple rooms available under the same listing, which would explain why they have listings getting 58.5 reviews in a span of 30 or 31 days. 

Graphs by Availabile Days out of 365 Days

In [ ]:
px.histogram(df, x = 'availability_365', title = 'Distribution of Available Days out of 365 days')

We can see a bimodal distribution with spkies at both ends. This tells us that on one extreme, most NYC hosts were not leasing out their place during 2019, or maybe for a duration not more than a week. On the other extreme, there were hosts that had listings available for almost the entire year of 2019. 

In [ ]:
sns.catplot(x='neighbourhood_group', y='availability_365', data = tidydf, hue='room_type')

All boroughs had availabilities rangine from 0 to 365, with not much of a differentiation between room types. It is interesting to note that a majority of Staten Island listings were available for more days of the year compared to other boroughs. 

## NLP

- Use nlp libraries to group names 
- find correlation to popularity maybe price

- Word differences from different room types 
- popularity: some how combine the popularity scores
- pricing 
- neighborhoods 

We will be exploring the name feature of the Airbnb dataset in depth using NLP techniques. With the use of the popular NLP library spacy we are able to clean the listing names and extract relevant information from them such as the most frequnctly used words in listings as well as non english symbols used in the listings. 

In our analysis of the names of the listings we provide visualizations to show the relationship of the name listings across price subgroups and popularity subgroups which we equated to be the number of reviews per month. 

Additionally we explore the unique listing names which contain non english symbols and plot their locations to see if there is any correlation between the language used in the listing and the listing location. This correlation may be present in listing names which contain chinese characters and are potentially located in china town.

In [ ]:
df_r_low = df[df['reviews_per_month'].between(0, 0.45)] #25%
df_r_mid = df[df['reviews_per_month'].between(0.46, 1.23)] #50%
df_r_high = df[df['reviews_per_month'].between(1.24, 2.68)] #75%
df_r_extra = df[df['reviews_per_month'].between(2.69, 70)] #75%

#Need better way of splitting the dataset based on quantiles maybe have list of quantiles and create functions 

#show the amount of listings with quantiles 

df_p_low = df[df['price'].between(0, 70)] #25%
df_p_mid = df[df['price'].between(70.01, 109)] #50%
df_p_high = df[df['price'].between(109.01, 175)] #75%
df_p_extra = df[df['price'].between(175, 9999)] #100%

In [ ]:
df_r_low.shape

In [ ]:
df_p_low.shape

In [ ]:
quartiles_r = df[["reviews_per_month"]].describe()

quartiles_p = df[["price"]].describe()

print(df_r_low.shape, '\n\n'
, quartiles_r, '\n\n', quartiles_p)

In [ ]:
#Word cloud of all not cleaned names 

df_nc = pd.read_csv('Airbnb_NYC_2019.csv')

from wordcloud import WordCloud

corpus = list(df_nc["name"].values)
clean_named = (' '.join(w for w in corpus if isinstance(w, str) ))

wordcloud = WordCloud(width = 1200, height = 600,
                background_color ='white',
                min_font_size = 10).generate(clean_named)

# plot the WordCloud image                       
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
df.columns

***Why are we using the cleaned dataset for names***

Reason: For the most part it takes out irrelevant listing which potenitally were not listing active in 2019. In terms of the focus of this report we ideally aim to see active listing in 2019 and compare the relationship between subgroups of price and popularity via number of review per month. 

Reason: we only use the cleaned dataset because we want to only look at names that resemble the listing for airbnb in newyork during 2019. The cleaned data removes all the data that is invalid.



In [ ]:
#Explore chinatown name find locations of the chinese names

df.shape

In [ ]:
# #COUNTS MOST POPULAR NAMES
# from collections import Counter

# #clean_named

# # split() returns list of all the words in the string
# split_it = clean_named.split()
  
# # Pass the split_it list to instance of Counter class.
# Counter = Counter(split_it)
  
# # most_common() produces k frequently encountered
# # input values and their respective counts.
# most_occur = Counter.most_common(30)
  
# print(most_occur)

In [ ]:
#RANDOM CODE GOT FROM CLASS
#dividing cars into 3 categories by weight
# min_price = min(df['weight'])
# max_price = max(df['weight'])
# per_bin = (max_wt - min_wt)/3

# bin1_start = min(df['weight'])
# bin1_end = bin1_start + per_bin
# bin2_end = bin1_end + per_bin

# def label_wt(row):
#     if bin1_start <= row['weight'] and row['weight'] <= bin1_end:
#         return 'light'
#     elif row['weight'] <= bin2_end:
#         return 'medium'
#     else:
#         return 'heavy'
# df['weight_categ'] = df.apply(lambda row: label_wt(row),axis=1)
# df.head()
# print("Thresholds: \nBin 1 = ",bin1_end,"\nBin 2 = ",bin2_end,"\nBin 3 = ",max_wt,"\nSize of each bin = ",per_bin)

In [ ]:
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(clean_names)

# threshold = 0.2

# for x in range(0,X.shape[0]):
#   for y in range(x,X.shape[0]):
#     if x!=y:
#       if cosine_similarity(X[x],X[y])>threshold:
#         print(df_1["name"][x],":",corpus[x])
#         #print(df_1["name"][y],":",corpus[y])
#         print("Cosine similarity:",cosine_similarity(X[x],X[y]))
#         print()

### Removing Puncuations

In [ ]:
### Creating a data frame that has only id and names.

#spacy import 
from spacy.lang.en import English
import spacy
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:

dfs = [df_r_low, df_r_mid, df_r_high, df_r_extra]

names_c = [] #list of all the names as strings

nlp = English()

nlp.Defaults.stop_words |= {"room", "bedroom", "apartment", "apt"} #added stop words 

for i in dfs: 

    names_low = i["name"].values

    clean_named = (' '.join(str(n).lower() for n in names_low) )

    name = nlp(clean_named) #tokinzed name

    filtered_name = []

    # filtering name from stop words
    for word in name:
        if word.is_stop==False and word.is_punct==False: #takes out punctuation 
            filtered_name.append(word)

    name_clean = ' '.join([str(w).lower() for w in filtered_name])

    names_c.append(name_clean)

#find most frequent words 
names_c
#create histogram plots and two way histogram plots 
# from collections import Counter

# def plot_names(names): 
#     split_it = names.split()
#     counts = Counter(split_it)
#     # most_common() produces k frequently encountered
#     # input values and their respective counts.
#     most_occur = counts.most_common(20)
#     most_df = pd.DataFrame(most_occur, columns =['Words', 'Freq'])
#     plt.figure()
#     sns.barplot(y="Words", x="Freq", data=most_df)


# #CREATES THE PLOTS
# for n in names_c: 
#     plot_names(n)


#WORDS CLOUDS

# wordcloud = WordCloud(width = 1200, height = 600,
#                 background_color ='white',
#                 min_font_size = 10).generate(name_clean)

# # plot the WordCloud image                       
# plt.figure(figsize = (8, 8), facecolor = None)
# plt.imshow(wordcloud)
# plt.axis("off")
# plt.tight_layout(pad = 0)
# plt.show()

In [ ]:
dfs_p = [df_p_low, df_p_mid, df_p_high, df_p_extra]

names_cp = [] #list of all the names as strings

nlp = English()

nlp.Defaults.stop_words |= {"room", "bedroom", "apartment", "apt"} #added stop words 

for i in dfs_p:

    names_low = i["name"].values

    clean_named = (' '.join(str(n).lower() for n in names_low) )

    name = nlp(clean_named) #tokinzed name

    filtered_name = []

    # filtering name from stop words
    for word in name:
        if word.is_stop==False and word.is_punct==False: #takes out punctuation 
            filtered_name.append(word)

    name_clean = ' '.join([str(w).lower() for w in filtered_name])

    names_cp.append(name_clean)

    #find most frequent words 

names_cp
# #create histogram plots and two way histogram plots 
# from collections import Counter

lowp = names_cp[0].split()
countlowp = Counter(lowp)
countlowp50 = countlowp.most_common(50)
lowpdf = pd.DataFrame(countlowp50, columns =['Words', 'Freq'])

midp = names_cp[1].split()
countmidp = Counter(midp) 
countmidp50 = countlowp.most_common(50)
midpdf = pd.DataFrame(countmidp50, columns =['Words', 'Freq'])

highp = names_cp[2].split()
counthighp = Counter(highp)
counthighp50 = counthighp.most_common(50)
highpdf = pd.DataFrame(counthighp50, columns =['Words', 'Freq']) 

extp = names_cp[3].split()
countextp = Counter(extp) 
countextp50 = countextp.most_common(50)
extpdf = pd.DataFrame(countextp50, columns =['Words', 'Freq'])

asd1 = pd.merge(lowdf,midpdf,on='Words',how='outer')
asd2 = pd.merge(asd1,highpdf,on='Words',how='outer')
asd3 = pd.merge(asd2,extpdf,on='Words',how='outer')

asd3


# def plot_names_perc(names): 
#     split_it = names.split()
#     counts = Counter(split_it)
#     # most_common() produces k frequently encountered
#     # input values and their respective counts.
#     most_occur = counts.most_common(20)
#     most_df = pd.DataFrame(most_occur, columns =['Words', 'Freq'])
#     plt.figure()
#     sns.barplot(y="Words", x="Freq", data=most_df)

# # for n in names_c: 
# #     plot_names_perc(n)

In [ ]:
#TFDIF , bag of words

#Parts of speech 

#Detecting language

#Plotting on map where the non english listing are 

#need to get the two way histogram for that

# Number of review, reviews per month, avalibilty 365. 
### How to develope a score for  popularity. 

#Default a reviews per month 

In [ ]:
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
import string

### Other way to clean the names

In [ ]:
cleaned_names = []


for i in list(df_r_extra["name"].values):
    text_tokens = word_tokenize(str(i).lower())
    clean_toks = [word for word in text_tokens if word not in nlp.Defaults.stop_words and word not in string.punctuation]
    i = ' '.join([str(n) for n in clean_toks])
    cleaned_names.append(i)

In [ ]:
cleaned_names

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

import multiprocessing
import pandas as pd
import numpy as np
from multiprocessing import Pool
import scipy.sparse as sp

num_cores = multiprocessing.cpu_count()
num_partitions = num_cores-2 # I like to leave some cores for other
#processes
print(num_partitions)

def parallelize_dataframe(df, func):
    a = np.array_split(df, num_partitions)
    del df
    pool = Pool(num_cores)
    #df = pd.concat(pool.map(func, [a,b,c,d,e]))
    df = sp.vstack(pool.map(func, a), format='csr')
    pool.close()
    pool.join()
    return df

def test_func(data):
    #print("Process working on: ",data)
    tfidf_matrix = TfidfVectorizer.transform(data["text"])
    #return pd.DataFrame(tfidf_matrix.toarray())
    return tfidf_matrix

#df = pd.DataFrame({'col': [0,1,2,3,4,5,6,7,8,9]})
#df =  data_pd
tf_idf = parallelize_dataframe(cleaned_names, test_func)



#tf_idf = TfidfVectorizer().fit_transform(cleaned_names)

# Use .A to display a sparse matrix.
(tf_idf * tf_idf.T).A

In [ ]:
from textblob import TextBlob

docs_df = pd.DataFrame(TextBlob(d).word_counts for d in cleaned_names).T
docs_df = docs_df.fillna(0)
docs_df.head()

tf = docs_df / docs_df.sum()
tf.head()

In [ ]:
#Finding non alphabet characters 

noneng = []


for i in list(df["name"].values): 
    a = str(i)
    if a.isascii():
        continue
    else: 
        noneng.append(i)
  

len(noneng)

In [ ]:
noneng

In [ ]:
import re

chineseLists = []

def isChinese(s):
    s = str(s)
    if len(re.findall(r'[\u4e00-\u9fff]+', s))>0:
        return True
    return False

for i in noneng: 
    a = str(i)
    if isChinese(a): chineseLists.append(i)

chineseLists


# sample = '法拉盛中心温馨两房两厅公寓｡近一切｡2 br close to everything'
# re.findall(r'[\u4e00-\u9fff]+', sample)

In [ ]:
from emoji import UNICODE_EMOJI

emojiLists = []

def is_emoji(s):
    s = str(s)
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)


for i in noneng: 
    a = str(i)
    if is_emoji(a): emojiLists.append(i)

emojiLists



# sample = '法拉盛中心温馨两房两厅公寓｡近一切｡2 br close to everything'
# re.findall(r'[\u4e00-\u9fff]+', sample)

In [ ]:
chinese_index = []
emoji_index = []

for i in list(df["name"].values): 
    a = str(i)
    if is_emoji(a) and not a.isascii() :
        emoji_index.append(True)
    else: 
        emoji_index.append(False)

for i in list(df["name"].values): 
    a = str(i)
    if isChinese(a) and not a.isascii() :
        chinese_index.append(True)
    else: 
        chinese_index.append(False)

df_noneng = df[chinese_index]
df_emoji = df[emoji_index] #logical indexing

Density Map of Listings with Emojis

In [ ]:
plt.figure(figsize=(15,30))
sns_map_emoji = sns.scatterplot(x='longitude', y='latitude',s=20, data=df_emoji)
ctx.add_basemap(sns_map_emoji, crs = 'EPSG:4326', source=ctx.providers.CartoDB.Positron)
sns_map_emoji.set_axis_off()

In [ ]:
name = "t #**3131 #s"


a = '❤️lovely historic brownstone charm❤️ legal listing'
b = '★★★★★- lux astoria |❤of nyc| near subway/manhattan'
c = '纽约之家'
d= 0.13124134134234

str(d).isascii()



#list(df_r_extra['name'].values)

Density Map for Non-English Listings

In [ ]:
plt.figure(figsize=(15,30))
sns_map_noneng = sns.scatterplot(x='longitude', y='latitude',s=20, data=df_noneng)
ctx.add_basemap(sns_map_noneng, crs = 'EPSG:4326', source=ctx.providers.CartoDB.Positron)
sns_map_noneng.set_axis_off()

In [ ]:
sns.countplot(y = 'neighbourhood_group', data= df_noneng).set_title("Listings by Borough for Noneng")

In [ ]:
sns.countplot(y = 'neighbourhood', data= df_noneng).set_title("Listings by Borough for Noneng")

In [ ]:
df_noneng

In [ ]:
sns.countplot(y = 'host_name', data= df_noneng).set_title("Host Name for Noneng")

In [ ]:
px.histogram(tidydf, x = 'price', title = 'Total Price Distribution')

In [ ]:
px.histogram(df_noneng, x = 'price', title = 'Non eng Price Distribution')

In [ ]:
df_noneng

## Other research questions:

Predicting Airbnb Prices with separate features and past data?

Is there a way to find the most optimal listing for a trip?

Want to buy a property using airbnb data which one will bring in the most profit depending on the existing data?